----

# Artificial Intelligence - COMPSCI4004 2024-2025

## Lab 8: Function approximations for RL

## RBF

----

This lab is a case-study where we attempt to apply function approximations to a specific problem namely an Open AI Gym problem called MountainCar.

This notebook relies on 
- The Open AI Gym environment which you must have installed to proceed (see Lab 1 or https://gym.openai.com/docs/).
- `Sklearn` from scikit (we will use the SGD and RBF function from `sklean` to define and fit our functions easily without having to write too much code ourselves for doing the gradient descent learning)


**Guide:** 

We recommend studying Q8.1-Q8.4 but skip Q8.5 and move on to Q8.6. You can return to Q8.5 at a later stage later (to fully understand the type of function we are using).

<font color=dark-magenta>TASK:</font> Tasks we recommend you consider before moving on (only a few are blocking).

<font color=dark-magenta>CHECKPOINTS:</font> Things you should take notice of before proceeding.

----

### Q8.1 Introduction & Housekeeping 

- Let's import what we need for this lab:

In [ ]:
# The basics
%matplotlib inline
import matplotlib.pyplot as plt
import time
import itertools
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import sys
import os
from collections import namedtuple

# Import the open AI gym
import gym

# Let's import basic tools for defining the function and doing the gradient-based learning
import sklearn.pipeline
import sklearn.preprocessing
#from sklearn.preprocessing import PolynomialFeatures # you can try with polynomial basis if you want (It is difficult!)
from sklearn.linear_model import SGDRegressor # this defines the SGD function
from sklearn.kernel_approximation import RBFSampler # this is the RBF function transformation method

print(sys.version)
print(os.path)
print("\n\nYou are good to go")

---

### Q8.2 The Mountain Car problem

#### Q8.2.1 The environment

- <font color=dark-magenta>TASK:</font> Familiarise yourself with the MountainCar problem here: https://gym.openai.com/envs/MountainCar-v0/ and here https://github.com/openai/gym/wiki/MountainCar-v0. The environment is specified by the following code https://github.com/openai/gym/blob/master/gym/envs/classic_control/mountain_car.py however it is not critical that you understand this in detail. Do a quick analysis of the task environment, specifically: 
     - what are the rewards ?
     - what is the performance measure?
     - are the states discrete or continuous?  
     - which actions can we take ?
     - are the actions discrete or continuous ?
     - what is the starting state (is it always the same, you can check with `env.reset()` followed by `env.state`)

<div style="border:2.0px solid red"></div>
<font color="red">SOLUTION</font>

- The performance measure is defined from the rewards 
- The rewards are -1 for all states (i.e. the agent should leave the environment as soon as possible)
- The states are continuous (velocity and position)
- The actions are discrete (push left, no push, push right)

<div style="border:2.0px solid red"></div>

----

#### Q8.2.2 Let's load the environment

In [ ]:
#env = gym.envs.make("MountainCar-v0")#, render_mode="rgb_array") # ignore a potential "gym.spaces.Box autodetected" warning
env = gym.envs.make("MountainCar-v0", render_mode="rgb_array") # use this if you are using the newer (eg 0.26.x) or older versions of the gym package!
env.reset()

- Let's take a quick look at the type of observations (of the states) and actions we can expect:

In [ ]:
observation_examples = np.array([env.observation_space.sample() for x in range(10000)])

In [ ]:
print(env.action_space)
print(env.observation_space)
print(observation_examples)

- <font color=dark-magenta>CHECKPOINT:</font> you should notice that the states are represented by continuous numbers and the actions are discrete {0,1,2}

----

### Q8.3 Designing the RL agent

To design a RL agent that can learn a good policy which will hopefully get us to the top, we'll - as for most RL-based agents - need:

* 1) a representation of Q(s,a) (or U(s)): we will opt for a function approximation to the Q(s,a) function (more on the specific choice later on)
<br>
* 2) a way to find the policy from the Q function Q(s, a): we know how to do this by taking the maximum Q(s,a) for a specific state s.
<br>
* 3) a way to update the Q(s, a) function: there are several ways to do this but we will opt for Q-leaning with function approximation, i.e. we follow the current policy with some exploration and update the parameters in the function approximation using gradient-base methods (see AIMA 21.4).
<br>

Furthermore, it would also be nice if we could have:

* 4) a way to visualize the policy in the env.

----

- <font color=dark-magenta>TASK:</font> Ad 1) $Q(s, a)$ representation: Explain why it would be problematic to use a Q-table to represent the Q values for each state-action pair?

<div style="border:2.0px solid red"></div>
<font color="red">SOLUTION</font>

- The main issue is that the states are continuous, i.e. the table would have infinitely many entries for the states. We could potentially quantize the states (we will leave that up to the reader to try out)

- Secondly, we would want to generalise from seen states to unseen states and a (predictive) function approximation will let us do that (assuming you choose a suitable function that can fit the observations while also providing some predictive power).

<div style="border:2.0px solid red"></div>

----

### Q8.4 Implementing the basics

#### Q8.4.1 Implementation

We will start by implementing the basics, namely 

- a method for defining a random function i.e. one that gives random values for Q(s,a) (we will later learn this function).
- a method to find a policy from a $Q(s,a)$.
- a method to execute/visualize the policy in the environment.

##### A function approximation class:
The idea is to lean a function for $Q(s,a)$, for now we can simply define a random Q(s,a) and see what happens:

In [ ]:
class MyRandomFunctionApproximator():
    def __init__(self):
        self.model=None
                
    def predict(self, s, a=None):
        q_s_a = np.random.randn(3,1) # defines the probablity for each action 
        return q_s_a
                          

- <font color=dark-magenta>TASK:</font> Instantiate a function (from MyRandomFunctionApproximator() ) and make predictions for the state (1.0, -0.8). 

In [ ]:
# Insert your code here

<div style="border:2.0px solid red"></div>
<font color="red">SOLUTION</font>

In [ ]:
my_random_func_approx = MyRandomFunctionApproximator()
q_s_a = my_random_func_approx.predict([0.0, -1.1])
q_s_a

- Note: the function is not dependent on s and simply outputs random Q values (i.e. an random agent).

<div style="border:2.0px solid red"></div>

----

##### A policy function
We want to be able to define a policy from the $Q(s,a)$ function. Let's define a class that can help us do that (notice that a handle to the function is returned).

In [ ]:
def create_policy(func_approximator, epsilon, nA):
    """
    Creates an greedy policy with the exploration defined by the epsilon and nA parameters
    
    Input:
        func_approximator: An estimator that returns q values for a given state
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.
    
    Output:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(state):
        """
        Input:
            state: a 2D array with the position and velocity
        Output:
            A,q_values: 
        """
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = func_approximator.predict(state)
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A,q_values  # return the potentially stochastic policy (which is due to the exploration)

    return policy_fn # return a handle to the function so we can call it in the future

- <font color=dark-magenta>TASK:</font> From the code above, determine how the policy finds a policy based on `func_approximator` for Q(s,a) (i.e., determines what to do in each state). In other words, we have a prediction for Q(s,a) - how does the function above determine the best action.


- <font color=dark-magenta>TASK:</font> Implement code to find the policy in state (-1.1,0.8) based on your random function approximation above with epsilon=0.0. Run it multiple times; why do you see different actions for the same state when you rerun it? Now run it with epsilon=0.05 - what changed?

*Hint* you need to instantiate an instance of `create_policy` with the proper arguments. This can be a bit tricky if you are not a Python expert - ask the tutors if you get stuck!

In [ ]:
# Insert your code here

<div style="border:2.0px solid red"></div>
<font color="red">SOLUTION</font>

It is done in a greedy way i.e. $a \leftarrow {\max _a}Q(s,a)$. 

<br>



In [ ]:
my_rnd_policy_fn = create_policy(my_random_func_approx, 0.0, env.action_space.n)

In [ ]:
act, q_vals = my_rnd_policy_fn((-1.1,0.8))
act

- The function approximator is random i.e. gives different Q-values which leads to a different policy

<div style="border:2.0px solid red"></div>

##### Execute the policy

Let's define a function for actually executing the policy in the Open AI gym environment:

In [ ]:
def exec_policy(env, func_approximator, verbose=False):
    """
        A function for executing a policy given the funciton
        approximation (the exploration is zero)
    """

    # The policy is defined by our function approximator (of the utility)... let's get a hdnle to that function
    policy = create_policy(func_approximator, 0.0, env.action_space.n)
            
    # Reset the environment and pick the first action
    state = env.reset()
                    
    # One step in the environment
    for t in itertools.count():
        #plt.imshow(env.render('rgb_array'))
        plt.imshow(env.render()) # use this if you are using the newer (eg 0.26.x) or older versions of the gym package!
        plt.show()

        # The policy is stochastic due to exploration 
        # i.e. the policy recommends not only one action but defines a 
        # distrbution , \pi(a|s)
        pi_action_state, q_values = policy(state)
        action = np.random.choice(np.arange(len(pi_action_state)), p=pi_action_state)
        #print("Action (%s): %s" % (action_probs,action)

        # Execute action and observe where we end up incl reward
        #next_state, reward, done, _ = env.step(action)
        next_state, reward, done, _, _ = env.step(action)   # use this if you are using the newer (eg 0.26.x) or older versions of the gym package!
        
        if verbose:
            print("Step %d/199:\n" % (t), end="")
            #print("\t state     : %s\n" % (state), end="")
            print("\t state     : %s\n" % (" ".join(str(s) for s in state)), end="")
            print("\t q_approx  : %s\n" % (q_values.T), end="")
            print("\t pi(a|s)   : %s\n" % (pi_action_state), end="")            
            print("\t action    : %s\n" % (action), end="")
            print("\t next_state: %s\n" % (next_state), end="")
            print("\t reward    : %s\n" % (reward), end="")                        
        else:
            print("\rStep {}".format(t), end="")
       
        if done:
            break
            
        state = next_state

Let' implement code for executing the policy and see the agent (in an external figure) running the env using the exec_policy method above. 

In [ ]:
a_random_func_approx = MyRandomFunctionApproximator()

In [ ]:
a_random_policy_fn = create_policy(a_random_func_approx,0.0,env.action_space.n)

Let's now execute the greedy policy. *NOTICE:* an external figure will open, and you'll need to switch to it rather quickly to see what's going on (you can rerun of course)

In [ ]:
exec_policy(env, a_random_func_approx, verbose=False)
env.close()

You *must* close the external figure using the `env.close()` command (do not try to do it manually!)

- <font color=dark-magenta>TASK/CHECKPOINT:</font> Did your agent reach the top? - probably not due to the random policy. Try to rerun the execution and observe the behavior. Change the `verbose=False` argument to true and observe the detailed trace. Make sure you understand what is being printed. Specifically, explain what "pi(s|a)" refers to (`pi_action_state` variable in the code).

In [ ]:
# Insert code with verbose=True here

<div style="border:2.0px solid red"></div>
<font color="red">SOLUTION</font>

In [ ]:
#env.close()
env.reset()

In [ ]:
exec_policy(env, a_random_func_approx, verbose=True)
env.close()

<div style="border:2.0px solid red"></div>

----

### Q8.5 Learning function approximators from observations

The policy which came from the random function approximator in the previous question was obviously not a good choice as it did not lead us to the goal.

We need to learn a better function approximator $Q(s,a)$ which ultimately defines the policy. In order to do so we need to go out into the environment and learn from the reward (and punishment) and from the way the env reacts to our actions.

<font color=dark-magenta>NOTICE:</font> You can skip Q8.5 if you just want to see the agent solve the problem.

----

#### Q8.5.1 Learning function approximators from observations

First, we will look closer at a particular type of functions, namely [radial basis functions](https://en.wikipedia.org/wiki/Radial_basis_function) and how we can lean them. We will do this in general setting, i.e., not specifically for RL and Q-learning.

The basic linear models we have seen in the lecture are of the form

$${f}({\mathbf{x}}) = w_0 + \sum\limits_{d=1}^{D} {{w_d}\cdot {x_d}} $$

This type of linear model can represent a straight line (in 1D) or a plane (in 2D) or a hyperplane in higher dimensions. However, we are often interested in functions which are not necessarily straight lines or hyperplanes. Linear models can easily be extended in this regard to support this. E.g. imagine a polynomial model in 1D, i.e. $f({x})=w_0+w_1\cdot x+w_2\cdot x^2$

In this part, we will generalize this idea to rather powerful models called radial basis functions:

$${f}({\mathbf{x}}) = \sum\limits_j^{} {{w_j}{\phi _j}\left( s \right)}  = \sum\limits_j^{} {\left( {{w_j} \cdot \exp \left( { - \tfrac{1}{{{\gamma_j^2}}}\left\| {{\mathbf{x}} - {{\boldsymbol{\mu }}_j}} \right\|}_2^2 \right)} \right)} $$

All we have done here is to transform the inputs using a set of (fixed) functions $\phi$). The basic model is the same i.e. we multiply some weight onto a version of x. Once you get the intuition they are quite easy to work with...

----

First, let understand what the $exp(\cdot)$ from above actually means:

In [ ]:
# a custom rbf function
def my_rbf(x,mu,gamma):
    return np.exp(- np.linalg.norm(x-mu,axis=-1)**2 / gamma**2)

In [ ]:
# an x range on which we want to evaluate the rbf function 
x = np.linspace(-10.0,10.0,1000)
x = np.expand_dims(x,1)

In [ ]:
# shape of x
x.shape

In [ ]:
# plo the function 
phi_test = my_rbf(x, 2.0, 2.3)
plt.plot(x,phi_test)
plt.xlabel("x")
plt.ylabel("y=f(x)")
plt.title("rbf - a bump")
plt.grid(True)

<font color=dark-magenta>TASK:</font> Change the shape and location of the bump funciton (i.e the RBF) by changing the gamme and mu parameter.

----

We can easily define multiple RBF with a different location ($\mu$) and scale ($\gamma$), i..e,

In [ ]:
# define a few RBF functions with different parameters
phi1 = my_rbf(x, 0.0, 1.3)
phi2 = my_rbf(x, 6.0, 4.3)
phi3 = my_rbf(x, -4.0, 3.3)

In [ ]:
# plot the rbf functions
plt.plot(x,phi1)
plt.plot(x,phi2)
plt.plot(x,phi3)
plt.legend(('$\phi_1(x;0,1.3)$','$\phi_2(x;6,4.3)$','$\phi_3(x;-4,3.3)$'))
plt.xlabel("x")
plt.ylabel("y=f(x)")
plt.title("bumps - unweighted")
plt.grid(True)

- And we can sum them together...

In [ ]:
# add the rbf function and plot them
y = np.sum([phi1,phi2,phi3],axis=0)
plt.plot(x,phi1)
plt.plot(x,phi2)
plt.plot(x,phi3)
plt.plot(x,y)
plt.legend(('$\phi_1(x;0,1.3)$','$\phi_2(x;6,4.3)$','$\phi_3(x;-4,3.3)$','$y=\phi_1(x)+\phi_2(x)+\phi_3(x)$'))
plt.xlabel("x")
plt.ylabel("y=f(x)")
plt.grid(True)
plt.title("RBF - weighted")

- <font color=dark-magenta>CHECKPOINT:</font> The sum of the individual bump function is in itself a function of the form: 
$${f(x)} = \sum\limits_j^{} {{w_j}{\phi _j}\left( x \right)}  = \sum\limits_j^{} {\left( {{w_j} \cdot \exp \left( { - \tfrac{1}{{{\gamma _j}}}\left\| {{{x}} - {{\mathbf{\mu }}_j}} \right\|^2} \right)} \right)}$$
where the weights are all 1.

Now, what happens if we change the weight (these are the ones we will learn later on)

In [ ]:
w = np.array((0.0,0.1,0.5,-0.4))
phi1w = w[1]*my_rbf(x, 0.0, 1.3)
phi2w = w[2]*my_rbf(x, 6.0, 4.3)
phi3w = w[3]*my_rbf(x, -4.0, 3.3)

In [ ]:
y = w[0]+np.sum([phi1w,phi2w,phi3w],axis=0)
plt.plot(x,phi1w)
plt.plot(x,phi2w)
plt.plot(x,phi3w)
plt.plot(x,y)
plt.xlabel("x")
plt.ylabel("y=f(x)")
plt.grid(True)
plt.title("RBF - weighted (with negative factor)")
plt.legend(('$\phi_1(x;0,1.3)$','$\phi_2(x;6,4.3)$','$\phi_3(x;-4,3.3)$','$y=\phi_1(x)+\phi_2(x)+\phi_3(x)$'))

- <font color=dark-magenta>CHECKPOINT:</font> Now the sum of the individual bump function is a function of the form: 
$${f(x)} = \sum\limits_j^{} {{w_j}{\phi _j}\left( x \right)}  = \sum\limits_j^{} {\left( {{w_j} \cdot \exp \left( { - \tfrac{1}{{{\gamma _j}}}\left\| {{{x}} - {{\mathbf{\mu }}_j}} \right\|^2} \right)} \right)}$$
where the weights are not all one. As you should see, this allows us to define a very flexible function by simply tweaking the weights a bit (we would want to learn these weights).


- <font color=dark-magenta>TASK OPTIONAL:</font> Try to extent the above example to 2D i.e. where the bumps are defined with two coordinates and visualise a function of your choice.


----

#### Q8.5.2 Specifying a model with randomly placed bumps in sklearn


To make life easier, we will use a toolbox for handling the RBF function in our actual agent, namely the `sklean` package.

In `sklean` we can define a set of RBF functions which are randomly located (i.e. the $\mu$ parameter is randomly chosen) as follows:

In [ ]:
feature_transformer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
feature_transformer.fit(x)

The above code will give us 400 $\phi$ functions all with different location, $\mu_i$, but some have the same variance,$\gamma_j$.

We can now transform our input $x$ (we compute the output of individual $\phi$ functions) as follows:

In [ ]:
# let's check the shape before transforming
x.shape

In [ ]:
x_transformed = feature_transformer.transform(x)
np.shape(x_transformed)

- <font color=dark-magenta>CHECKPOINT:</font> You should notice that our 1000 1D observations, x, has now been transformed into 400 dimensions corresponding to the output of each individual RBF function.

In [ ]:
# we can visualize our 1000 data ponts as a heat map (it is (almost) art!)
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(1,2,1)
ax0.imshow(np.repeat(x,100,axis=1)) # hack, ignore this just to make the plot pretty
ax0.set_xticks([50]) # hack, ignore this just to make the plot pretty
ax0.set_xticklabels("0") # hack, ignore this just to make the plot pretty
ax0.set_title("Org data (1000x1)")

ax1 = fig.add_subplot(1,2,2)
cmx = ax1.imshow(x_transformed)
ax1.set_title("Transformed data (1000x400)")
fig.colorbar(cmx)

----

#### Q8.5.3 Fitting our RBF model

Assume we have observed the following dataset

In [ ]:
x_train = np.array((-8.7,-6.5,-3.4,0.0,0.1,1.4,2.6,4.5)).reshape(-1, 1)
x_train_transformed = feature_transformer.transform(x_train)

y_train = np.array((3.4,4.4,1.1,-1.0,-1.5,4.4,6.6,-14.5))

In [ ]:
plt.scatter(x_train,y_train, color='r',marker='x')
plt.xlabel("x")
plt.ylabel("y=f(x)")
plt.title("training data")
plt.grid(True)
plt.legend("Training data")


- We now need to fit the weight in our model. This is as easy as for the basic model considered in the lecture, i.e. we what to find the 400 $w$'s in our model (the gamma and mu's are fixed):

$$f(x) = \sum\limits_{j = 1}^{400} {\left( {{w_j} \cdot \exp \left( { - \tfrac{1}{{{\gamma _j}}}\left\| {x - {\mu _j}} \right\|} \right)} \right)} $$

- We will use a Loss function similar to the one in the lecture
$$Loss({\mathbf{w}};{\{ {(x_n},{y_n})^{n = 1:8}\} }) = \sum\limits_{n = 1}^8 {{{\left( {{y_n} - \sum\limits_{j = 1}^{400} {\left( {{w_j} \cdot \exp \left( { - \tfrac{1}{{{\gamma _j}}}\left\| {{x_n} - {\mu _j}} \right\|} \right)} \right)} } \right)}^2}} + \lambda h({\mathbf{w}})$$
although there are few socalled regularization terms (summarised by $h(w)$) which helps us numerically solve the problem and not overfit to the observed data (this is not important in the current context).

- The optimization problem then becomes

$$\mathop {{{\mathbf{w}}^*} = \arg \min }\limits_w Loss({\mathbf{w}};{\{ ({x_n},{y_n})^{n = 1:8}\} })$$

- To solve the optimisaiton problem we will use a gradient based optimiser from sklearn called [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html) (you can look up the many parameters in this method if you want to know all the details)

Now putting this together is relatively simple in Python, i.e. we first define a "model" as follows using the [`SGDRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html), this function has several parameters which can be *fine-tuned* for your purpose. The most important thing is that it uses a squared loss (plus a small penalty for having large weights; this keeps the learning robust).

In [ ]:
model = SGDRegressor(learning_rate="constant",tol=1e-5,max_iter=10000)

Running the optimizaiton process to find the weights:

In [ ]:
model.fit(x_train_transformed,y_train)

We can now make predictions for any data, e.g. for our observed data points to see if the model can represent that funciton:

In [ ]:
y_hat_train = model.predict(x_train_transformed)

... and for all other points (unseen)

In [ ]:
x_demo = np.linspace(-10,10,100).reshape(-1,1)
x_demo_transformed = feature_transformer.transform(x_demo)
y_hat_demo = model.predict(x_demo_transformed)

In [ ]:
plt.scatter(x_train,y_hat_train,marker='x',color='red')
plt.plot(x_demo,y_hat_demo,marker='', color='blue')
plt.legend(('y predictions','y training observations'))
plt.xlabel("x")
plt.ylabel("y=f(x)")
plt.title("fitted RBF function")
plt.grid(True)

- <font color=dark-magenta>CHECKPOINT:</font> You should see that the function can perfectly fit the observed data (i.e. x_train, y_train). This is to be expected because we have a very flexible function and only very few training/observed data points. Usually, we would want to reduce the flexibility of the function a bit in this partiular case, but it serves to demo that we can easily fit complicated functions using the RBF model.

***Summary:*** We now have a very flexible non-linear function (still a linear model!) function that we can potentially use to fit the Q(s,a) function. We will see how to do that next.

----

### Q8.6 Function Approximations and Q-learning

In order to learn a good policy we need (at least) two things:

- a model i.e. a function approximation for $Q(s,a)$
- a q-learning agent which can update the $Q(s,a)$ function by exploring the environment.

#### Q8.6.1 Defining the model for Q(s,a)

We will set up a RBF regression model based on the ideas in Q8.5 using `sklean`.

An additional technicality: we need to consider that most optimization methods perform better if the input values (i.e., x) are standardised/normalised, i.e., have zero mean and one std deviation. We will set up a sklean function to do this. This is not important to the general problem here but something that improves performance.

In [ ]:
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)

In [ ]:
feature_transformer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
feature_transformer.fit(scaler.transform(observation_examples))

Let's have a look at the transformed values

In [ ]:
s_scaled = scaler.transform([observation_examples[0]])
print(s_scaled)

In [ ]:
s_transformed = feature_transformer.transform(s_scaled)
print(np.shape(s_transformed))

I.e. one state, s (we used x in Q8.5) is now represented by 400 values. 

----

#### The Function Approximator Implementation

We will define a class to handle our specific function approximator for the Q(s,a) function

In [ ]:
class FunctionApproximator():
    """
    Q(s,a) function approximator. 

    it uses a specific form for Q(s,a) where seperate functions are fitteted for each 
    action (i.e. four Q_a(s) individual functions)

    We could have concatenated the feature maps with the action TODO TASK?

    """
 
    def __init__(self, eta0= 0.01, learning_rate= "constant"):
        #
        # Args:
        #   eta0: learning rate (initial), default 0.01
        #   learning_rate: the rule used to control the learning rate;
        #   see https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html for details
        #        
        # We create a seperate model for each action in the environment's
        # action space. Alternatively we could somehow encode the action
        # into the features, but this way it's easier to code up and understand.
        #
        #
        self.eta0=eta0
        self.learning_rate=learning_rate
        
        self.models = []
        for _ in range(env.action_space.n):

            # You may want to inspect the SGDRegressor to fully understand what is going on
            # ... there are several interesting parameters you may want to change/tune.
            model = SGDRegressor(learning_rate=learning_rate, tol=1e-5, max_iter=1e5, eta0=eta0)
            
            # We need to call partial_fit once to initialize the model
            # or we get a NotFittedError when trying to make a prediction
            # This is quite hacky.
            model.partial_fit([self.featurize_state(env.reset())], [0])
            self.models.append(model)
    
    def featurize_state(self, state):
        """
        Returns the featurized representation for a state.
        """
        s_scaled = scaler.transform([state])
        s_transformed = feature_transformer.transform(s_scaled)
        return s_transformed[0]
    
    def predict(self, s, a=None):
        """
        Makes Q(s,a) function predictions.
        
        Args:
            s: state to make a prediction for
            a: (Optional) action to make a prediction for
            
        Returns
            If an action a is given this returns a single number as the prediction.
            If no action is given this returns a vector or predictions for all actions
            in the environment where pred[i] is the prediction for action i.
            
        """
        features = self.featurize_state(s)
        if a==None:
            return np.array([m.predict([features])[0] for m in self.models])
        else:            
            return self.models[a].predict([features])[0]
    
    def update(self, s, a, td_target):
        """
        Updates the approximator's parameters (i.e. the weights) for a given state and action towards
        the target y (which is the TD target).
        """
        features = self.featurize_state(s)
        self.models[a].partial_fit([features], [td_target]) # recall that we have a seperate funciton for each a 


<br>

<br>

* <font color=dark-magenta>TASK:</font> Inspect the code (e.g. add comments) and define specifically how the Q(s,a) function is actually implemented (hint: figure out how we input - or not - the action $a$ to the Q(s,a) function)? 

<div style="border:2.0px solid red"></div>
<font color="red">SOLUTION</font>

- The implementation uses separate functions for each action i.e. the action serves only as a selector for a set of three independent functions. I.e. we could write $Q_{w_a}(s)$ to indicate that we have separate functions with different parameter sets.

- Ideally, we would like to exploit the fact that there may be a correlation but that's beyond the scope of this lab.

<div style="border:2.0px solid red"></div>

#### Q8.6.2 A Q-learning Agent

The remaining thing to setup is the Q-learning agent that can follow a policy and update the function approximator.

In [ ]:
## 

# Keep track of some stats
EpisodeStats = namedtuple("Stats",["episode_lengths", "episode_rewards"])

# Main Q-learner
def q_learning(env, func_approximator, num_episodes, discount_factor=1.0, epsilon=0.1, epsilon_decay=1.0):
    """
    Q-Learning algorithm for Q-learning using Function Approximations.
    Finds the optimal greedy policy while following an explorative greedy policy.
    
    Args:
        env: OpenAI environment.
        func_approximator: Action-Value function estimator
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        epsilon: Exploration strategy; chance the sample a random action. Float between 0 and 1.
        epsilon_decay: Each episode, epsilon is decayed by this factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    
    # Keeps track of useful statistics
    stats = EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    for i_episode in range(num_episodes):
        
        # Create a handle to the policy we're following (including exploration)a
        policy = create_policy(
            func_approximator, epsilon * epsilon_decay**i_episode, env.action_space.n)
        
        # Print out which episode we're on, useful for debugging.
        # Also print reward for last episode
        last_reward = stats.episode_rewards[i_episode - 1]
        sys.stdout.flush()
        
        # Reset the environment and pick the first action
        state = env.reset()
                 
        # One step in the environment
        for t in itertools.count():
                        
            # Choose an action to take                        
            action_probs, q_vals = policy(state)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            
            # Take a step
            next_state, reward, done, _ = env.step(action)
    
            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t
            
            # TD Update
            q_values_next = func_approximator.predict(next_state)
            
            # Q-Value TD Target
            td_target = reward + discount_factor * np.max(q_values_next)
              
            # Update the function approximator using our target
            func_approximator.update(state, action, td_target)
            
            print("\rStep {} @ Episode {}/{} ({})".format(t, i_episode + 1, num_episodes, last_reward), end="")
                
            if done:
                break
                
            state = next_state


    return stats

<br>

<br>
- <font color=dark-magenta>TASK:</font> Inspect the code and determine the update strategy used by the agent (see AIMA 21.4). Specifically explain how this aligns with the gradients updates covered in the lecture and book (21.4). Note this is a fundamental question in today's lab and it is important that you understand it.

<div style="border:2.0px solid red"></div>
<font color="red">SOLUTION</font>

Reall the assumption in TD learning, if we take action $a$ in state $s$ and observe that we end up in$ s'$ AND we assume that this happens all the time, then the Bellman equations reduce to a very simple form, i.e.

$$Q(s,a) = R(s) + \gamma {\max _{a'}}Q(s',a')$$

This should hold true under the aforementioned assumptions! If it doesn't we can formulate a simple update rule to make it a "better" which leads us to the well-known Q-learning updates:

$$Q(s,a) \leftarrow Q(s,a) + \alpha ( R(s) + \gamma {\max _{a'}}Q(s',a') - Q(s,a) )$$

This can easily be computed for a tabular representation of Q. However, here we have a function approximator for Q, i.e. $\hat Q_w(s,a)$ which means that 

$$\hat Q_w(s,a) = R(s) + \gamma {\max _{a'}}\hat Q_w(s',a')$$,

i.e., we can not directly update the values of Q as these are controlled by the parameters $w$.

For the function approx case we can instead formulate the problem as a least squares problem i.e. try to minimize the distance between the just observed transition leading to the td-target: $$\text{TD-target}=R(s) +  \gamma {\max _{a'}}\hat Q_w(s',a')$$

Thus,
$$w^* = argmin_w (R(s) + \gamma {\max _{a'}}\hat Q_w(s',a') - \hat Q_w(s,a) )^2$$

This allows us to do an update based on the single observation of what happened in state $s$ taking action $a$. This can be done by computing the gradient of the loss wrt $w$ and making a small update to the weights. Since this is only based on one observation and there is some uncertainty to it we call it an online/stochastic gradient. The SDGRegression function used in the code does exactly this in the `FunctionApproximator.update` function - and it is exactly the same as AIMA 21.13 if you compute the gradients of the above loss and apply it manually with a certain step-size.

<div style="border:2.0px solid red"></div>

------